In [50]:
import pdb
%pdb

Automatic pdb calling has been turned ON


In [100]:
print?

In [42]:

from __future__ import (unicode_literals, absolute_import, division,
                        print_function)
import six

import os

def read_log_from_script(path_to_log):
    """
    Parse the log that is output from the `dev-build` script

    Parameters
    ----------
    path_to_log : str
        The path to the log file that results from `bash dev-build > log 2>&1`

    Yields
    ------
    package : str
        The name of the package that is being built
    output : list
        The lines that were output for the build/test/upload of `package`
    """
    TARGET_LINE = '/tmp/staged-recipes'
    full_path = os.path.abspath(path_to_log)
    output = []
    package_name = ''
    with open(full_path, 'r') as f:
        for line in f.readlines():
            # remove white space and newline characters
            line = line.strip()
            if line.startswith(TARGET_LINE):
                # always have to treat the first package differently...
                if package_name != '':
                    yield package_name, output
                package_name = os.path.split(line)[1]
                output = []
            else:
                output.append(line)

In [107]:
def parse_conda_build(lines_iterable):
    """
    Group the output from conda-build into
    - 'build_init'
    - 'build'
    - 'test'
    - 'upload'
    """
    from collections import defaultdict
    bundle = []
    end_section = iter(["BUILD START", "TEST START", "TEST END", "no end to this section"])
    next_line_might_be_test = False
    init = True
    build = False
    test = False
    key = 'init'
    ret = defaultdict(list)
    for line in lines_iterable:
        ret[key].append(line)
        # init
        if init:
            if line.startswith("BUILD START"):
                init = False
                build = True
                key = 'build'
        # build
        if build:
            if line.startswith("BUILD END"):
                next_line_might_be_test = True
                build = False
        # determine if test or upload comes next
        if next_line_might_be_test:
            if line.startswith("TEST START"):
                test = True
                key = 'test'
            else:
                key = 'upload'
        # test
        if test:
            if line.startswith("TEST END"):
                test = False
    return ret

In [108]:
log = 'build.log'
gen = list(read_log_from_script(log))
parsed = {name: parse_conda_build(lines) for name, lines in gen}
for name, groups in sorted(parsed.items()):
    print('{:20s} -- {}'.format(name, list(groups.keys())))

album                -- ['build', 'test', 'init', 'upload']
analysis             -- ['build', 'init', 'upload']
args                 -- ['build', 'init', 'upload']
autoconf             -- ['build', 'test', 'init', 'upload']
automake             -- ['build', 'test', 'init', 'upload']
bluesky              -- ['build', 'test', 'init', 'upload']
boltons              -- ['build', 'test', 'init', 'upload']
channelarchiver      -- ['build', 'test', 'init', 'upload']
collection           -- ['build', 'init', 'upload']
conda_etc            -- ['build', 'init', 'upload']
config-amx           -- ['build', 'init', 'upload']
config-chx           -- ['build', 'init', 'upload']
config-csx1          -- ['build', 'init', 'upload']
config-csx2          -- ['build', 'init', 'upload']
config-dev           -- ['build', 'init', 'upload']
config-hxn           -- ['build', 'init', 'upload']
config-ixs           -- ['build', 'init', 'upload']
config-lix           -- ['build', 'init', 'upload']
databroker      

In [110]:
parsed['pymongo']

defaultdict(list,
            {'build': ['Fetching package metadata: ........',
              'Solving package specifications: ........../tmp/root/ramdisk/mc/lib/python3.5/site-packages/requests/packages/urllib3/connection.py:266: SubjectAltNameWarning: Certificate for pergamon.cs.nsls2.local has no `subjectAltName`, falling back to check for a `commonName` for now. This feature is being removed by major browsers and deprecated by RFC 2818. (See https://github.com/shazow/urllib3/issues/497 for details.)',
              'SubjectAltNameWarning',
              '',
              'WARNING: conda-build appears to be out of date. You have version 1.18.1 but the',
              'latest version is 1.18.2. Run',
              '',
              'conda update -n root conda-build',
              '',
              'to get the latest version.',
              '',
              'Could not connect to https://pypi.python.org/packages/source/p/pymongo/pymongo-2.9.1.tar.gz',
              'Error: Connectio